在上一个笔记本中已经初步处理好了需要的全量数据，现在通过scikit-learn中的一些库函数对于借阅记录进行特征编码等，最终生成的numpy可读文件data_process.npy包含(保存在./data/processed/data_process.npy中)

1.所有物品的原始特征集item_set

2.所有用户的原始特征集user_set

3.训练集/验证集/测试集 特征向量 x_train/x_val/x_test

4.训练集/验证集/测试集 标签 y_train/y_val/y_test

In [1]:
import pandas as pd

time = 'LOAN_DATE'
Data = pd.read_csv("../data/processed/ZJULibrary2013_2019.csv", index_col = 0, dtype= str, nrows = 10000)
Data[time]=Data[time].astype(int)      # 先读取数据

In [2]:
from sklearn.preprocessing import LabelEncoder

user_features = ['PATRON_ID', 'STUDENT_GRADE', 'PATRON_DEPT', 'PATRON_TYPE']
# 用户侧特征       用户id          年级            学生学院       学生类型
item_features = ['ITEM_ID', 'SUBLIBRARY', 'CALLNO1','CALLNO2', 'PUBLISH_YEAR', 'AUTHOR', 'TITLE', 'PRESS']
# 物品侧特征       记录号      馆藏地         大类1      小类2       出版年          作者       题目    出版社
features = user_features + item_features

feature_max_idx = {}    # 记录每个特征共有多少种
for feature in features:
    lbe = LabelEncoder()
    Data[feature] = lbe.fit_transform(Data[feature]) + 1  # 从1开始
    feature_max_idx[feature] = Data[feature].max() + 1 # 防止padding后造成分类标签越界
    if feature == 'PATRON_ID':
        user_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(lbe.classes_)}
    if feature == 'ITEM_ID':
        item_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(lbe.classes_)}

In [3]:
Data.sample(3)

,PATRON_ID,STUDENT_GRADE,PATRON_DEPT,PATRON_TYPE,ITEM_ID,SUBLIBRARY,PUBLISH_YEAR,AUTHOR,TITLE,PRESS,LOAN_DATE,CALLNO1,CALLNO2
5347,168,1,11,3,4324,11,55,3294,6596,363,20130724,26,25
939,64,1,25,4,224,6,49,4520,3282,866,20130104,6,65
6308,11,1,32,5,7203,23,59,4678,5871,926,20130121,18,61


In [7]:
import numpy as np
import os
np.save(os.path.join('../data/processed/', "raw_id_maps.npy") ,np.array((user_map, item_map), dtype=object))

运行上方代码后会在./data/processed文件夹下生成储存数据代号标签与原数据的字典集raw_id_maps.npy

In [8]:
feature_max_idx  # 可以观察每一个字段有几类不同值，统计用户人数与图书本树

{'PATRON_ID': 245,
 'STUDENT_GRADE': 4,
 'PATRON_DEPT': 43,
 'PATRON_TYPE': 11,
 'ITEM_ID': 9443,
 'SUBLIBRARY': 28,
 'CALLNO1': 60,
 'CALLNO2': 120,
 'PUBLISH_YEAR': 67,
 'AUTHOR': 7328,
 'TITLE': 8441,
 'PRESS': 958}